In [2]:
#traing data
training_data = [
    ["Green", 3, 'Mango'],
    ['yellow', 3, 'Mango'],
    ['red', 1, 'grape'],
    ['red', 1, 'grape'],
    ['yellow', 3, 'lemon']
]

In [38]:
#header for the traing set
head = ['color', 'diameter', 'label']

In [4]:
#finding the unique for a colum in dataset
def unique_vals(rows,col):
    return set([row[col] for row in rows])

In [5]:
unique_vals(training_data,1)

{1, 3}

In [6]:
#count the number of each type of example in a dataset.
def class_counts(rows):
    counts ={}
    for row in rows:
        #in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return(counts)

In [7]:
class_counts(training_data)

{'Mango': 2, 'grape': 2, 'lemon': 1}

In [8]:
#test if a value is numeric
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [9]:
is_numeric(7)

True

In [10]:
is_numeric("red")

False

In [39]:
#a question is used to partition a dataset.
class Question:
    """"
    This class just records c 'cloumn number' (e.g,0 for color) and a 'column value' (Green). the 'match' method is used to compare
    the features value in an example to the features value stored in the question.
    """
    def __init__(self,column,value):
        self.column = column
        self.value = value
    def match(self,example):
        #compare the features value in example
        #feature value in this question
        val = example[self.column]
        if is_numeric(val):
            return val >=self.value
        else:
            return val == self.value
    def __repr__(self):
        #this is just a hepler method to print
        #question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "is %s %s %s?" % (
        head[self.column], condition,str(self.value))

In [12]:
#partion the dataset
def partition(rows,question):
    """
    for each row in the dataset,check if it matches the question.if 
    so,add it to 'true_row' , othewise,add it to 'flase rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [13]:
true_rows,flase_rows = partition(training_data, Question(0,'red'))

In [14]:
print(true_rows)

[['red', 1, 'grape'], ['red', 1, 'grape']]


In [15]:
print(flase_rows)

[['Green', 3, 'Mango'], ['yellow', 3, 'Mango'], ['yellow', 3, 'lemon']]


In [16]:
#caluculate the gini impurity for a list of rows.
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for l in counts:
        prob_of_l = counts[l]/float(len(rows))
        impurity -= prob_of_l**2
    return(impurity)

In [17]:
gini(training_data)

0.6399999999999999

In [18]:
#calculate the information gain
#the uncertinity of the starting node, minus the weighted impurity of two child nodes.
def info_gain(left, right,current_uncertanity):
    p = float(len(left)) /(len(left) + len(right))
    return current_uncertanity - p * gini(left) - (1 -p) * gini(right)
    

In [19]:
c = gini(training_data)


In [20]:
c

0.6399999999999999

In [24]:
t_r , f_r = partition(training_data,Question(0,'Green'))

In [25]:
t_r

[['Green', 3, 'Mango']]

In [26]:
f_r

[['yellow', 3, 'Mango'],
 ['red', 1, 'grape'],
 ['red', 1, 'grape'],
 ['yellow', 3, 'lemon']]

In [27]:
info_gain(t_r , f_r , c)


0.1399999999999999

In [28]:
tr , fr = partition(training_data,Question(0,'red'))

In [29]:
tr


[['red', 1, 'grape'], ['red', 1, 'grape']]

In [34]:
n_features = len(training_data[0])
n_features

3

In [30]:
fr

[['Green', 3, 'Mango'], ['yellow', 3, 'Mango'], ['yellow', 3, 'lemon']]

In [31]:
info_gain(tr,fr,c)

0.37333333333333324

In [35]:
#finding the best question to ask by iterating over every features/value and caculation the information
def find_best_split(rows):
    best_gain = 0 #keep track of the best information gain.
    best_question = None #keep train of the features / value that produced it 
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1 #number of columns
    for col in range(n_features): #for each features
        values = set(row[col] for row in rows)
        for vals in values: #for each feautres
            question = Question(col , vals)
            
            #try to split the datasets
            tr, fr = partition(rows, question)
            
            #skip the partion if any of tr or fr have no value
            if len(tr) == 0 or len(fr) == 0:
                continue
            
            #calculate the information gain from from this split
            gain = info_gain(tr, fr, current_uncertainty)
            
            #you actually can use '>' instead of '>=' here
            #but i wanted the tree to look a certain way for our 
            #toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question
    
    return best_gain , best_question


In [40]:
find_best_split(training_data)

(0.37333333333333324, is diameter >= 3?)

In [41]:
#a leaf node classifies data.  
#this holds a dictionary of class (eg "Mango") -> number of times it appear in the row 
#from the training data that reach this leaf

class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)

In [51]:
# A Decision Node ask a Question 
#this hold the reference to the question, and to the tow child nodes.

class Decision_node:
    
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        

In [52]:
#build the tree.

def build_tree(rows):
    
    #try partion the dataset of each of the uqique attribute
    #calculated  the information gain and return the question that produces the higest gain.
    gain, question = find_best_split(rows)
    
    #base case :no further info gain
    #scince we ask no further questions, we return a leaf.
    if gain == 0:
        return Leaf(rows)
    # if we reach here, we have found a useful features / value to partion on.
    true_rows, false_rows = partition(rows, question)
    
    # recursively build the tree branch.
    true_branch = build_tree(true_rows)
    
    # recursively build the false branch.
    false_branch = build_tree(false_rows)
    
    #return a question node.
    # this records the best features / value to ask at this point,as well as the branches to fallow
    # depending on the answer.
    
    return Decision_node(question, true_branch, false_branch)

In [46]:
def print_tree(node, spacing = ""):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "predict", node.predictions)
        return
    # Print the Question at this node
    print(spacing + str(node.question))
    
    # call this function recursively on the true branch
    print(spacing + '--> True:')
    print_tree(node.true_branch , spacing + " ")
    
    #class this function recursively on the false branch
    print(spacing + '--> False:')
    print_tree(node.flase_branch, spacing + " ")

In [56]:
def classify(row, node):
    
    #base case: we've reached a leaf 
    if isinstance(node, Leaf):
        return node.predictions
    # Decide whether to follows the true_branch or the false-branch. compare the features / value in the node,
    #to the example we're consideering.
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)
    

In [53]:
my_tree = build_tree(training_data)

In [54]:
my_tree


In [57]:
classify(training_data[0],my_tree)

{'Mango': 1}

In [62]:
#print the prediction at the leaf.

def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs = {}
    for l in counts.keys():
        probs[l] = str(int(counts[l]) / total * 100) + "%"
    return probs

In [63]:
print_leaf(classify(training_data[0],my_tree))

{'Mango': '100.0%'}